# DefaultAzureCredential을 사용한 인증 문제 해결

## 개요

DefaultAzureCredential은 Azure 애플리케이션에서 인증을 처리하는 데 권장되는 방법입니다. 이 방법은 여러 자격 증명 유형을 성공할 때까지 순차적으로 시도하여 간소화된 인증 흐름을 제공합니다. 이 notebook은 일반적인 인증 문제를 해결하고 적절한 설정을 보장하는 데 도움이 됩니다.

## DefaultAzureCredential 알아보기

DefaultAzureCredential은 다음 순서로 인증 방법을 시도합니다:

1. 환경 자격 증명
2. 워크로드 아이덴티티(Kubernetes 에서)
3. 매니지드 아이덴티티
4. Azure CLI 자격 증명
5. Azure PowerShell 자격 증명
6. Visual Studio Code 자격 증명
7. 대화형 브라우저 인증(fallback 형태)

## 전제 조건

다음이 설치되어 있는지 확인합니다:
- Azure CLI
- Azure Developer CLI (선택 사항)
- Python Virtual Environment 또는 Conda (`uv venv` 또는 `conda create` 을 사용)
- 필수 역할 할당 (Azure AI Developer)
- Jupyter Notebook 환경 - 파이썬 3.8 이상을 사용하도록 구성된 커널

## 인증 방법

### 1. Azure CLI 사용(로컬 개발에서 권장)

In [ ]:
# Install required packages
!pip install azure-identity

# 먼저 Azure CLI를 사용하여 인증합니다.
이는 로컬에서 개발을 할 때 권장되는 접근 방식입니다.

아래 코드를 실행하면 다음으로 리디렉션됩니다:
- 브라우저에서 Azure 포털을 사용하여 로그인을 완료
- 또는, 이미 컴퓨터에 로그인한 경우 Windows 로그인을 사용

이 코드는 다음과 같습니다다:
1. .env 파일에서 TENANT_ID를 포함한 환경 변수를 로드
2. Azure CLI를 사용하여 특정 테넌트에 로그인
3. 토큰을 가져와서 인증 테스트를 수행


In [ ]:
# Import required packages
from IPython.display import display
from IPython.display import HTML
import getpass
from dotenv import load_dotenv
import os
from pathlib import Path  # For cross-platform path handling

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv(parent_dir / '.env')  # Load environment variables from .env file

# Get tenant ID from environment variable
tenant_id = os.getenv("TENANT_ID")

# Azure login with specific tenant
!az login --tenant {tenant_id}

# Get subscription ID from connection string
conn_str = os.getenv("PROJECT_CONNECTION_STRING")
subscription_id = conn_str.split(';')[1] if conn_str else None

if subscription_id:
    # Set the subscription
    !az account set --subscription {subscription_id}
    print(f"✓ Successfully set subscription: {subscription_id}")
else:
    print("⚠️ Could not get subscription ID from PROJECT_CONNECTION_STRING")


# 다음으로 DefaultAzureCredential을 사용하여 토큰을 가져와서 인증을 테스트해 보겠습니다.

DefaultAzureCredential은 다음 순서로 여러 인증 방법을 시도합니다:
1. 환경 자격 증명 (환경 변수가 설정된 경우)
2. 매니지드 아이덴티티 자격 증명 (Azure에서 실행 중인 경우)
3. 공유 토큰 캐시 자격 증명 (이전 로그인에서) 
4. Visual Studio Code 자격 증명 (VS Code를 사용하는 경우)
5. Azure CLI 자격 증명 (방금 설정한 것)

다음 코드는 다음을 수행합니다:
1. DefaultAzureCredential 인스턴스 만들기
2. Azure Cognitive Services용 토큰을 가져오기 위해 시도
3. 토큰을 가져오면 성공 메시지를 출력합니다

>참고: 여러가지 다른 인증 방법을 시도하기 때문에 중간에 경고/오류 메시지가 표시될 수 있습니다.
>이것은 정상이며 마지막에 "Successfully acquired token!" 메시지만 보이면 나머지 경고/오류는 무시해도 됩니다.


In [ ]:
# Then use DefaultAzureCredential in your code
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AccessToken
import logging

# Enable detailed logging
logging.basicConfig(level=logging.DEBUG)

try:
    credential = DefaultAzureCredential()
    # Test token acquisition
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    print("Successfully acquired token!")
except Exception as e:
    print(f"Authentication failed: {str(e)}")

## 이제 인증에 성공했으므로, [2-environment_setup.ipynb](2-environment_setup.ipynb)로 진행하거나, 아래의 추가 인증 방법 또는 문제 해결 방법을 시도할 수 있습니다.


### 2. Visual Studio Code 사용 (선택 사항)

VS Code 에서 Azure Extension 을 함께 사용하고 있을 때:

In [ ]:
from azure.identity import DefaultAzureCredential, VisualStudioCodeCredential

try:
    # Explicitly try VS Code credentials
    vscode_credential = VisualStudioCodeCredential()
    token = vscode_credential.get_token("https://cognitiveservices.azure.com/.default")
    print("Successfully authenticated with VS Code credentials!")
except Exception as e:
    print(f"VS Code authentication failed: {str(e)}")

### 3. Service Principal 사용 (선택 사항 - 환경 변수를 설정해야 함)

In [ ]:
import os

# Set these environment variables before running your application
required_env_vars = {
    "AZURE_CLIENT_ID": "your-client-id",
    "AZURE_CLIENT_SECRET": "your-client-secret",
    "AZURE_TENANT_ID": "your-tenant-id"
}

# Verify environment variables are set
def check_env_vars():
    missing_vars = [var for var, _ in required_env_vars.items() 
                   if not os.getenv(var)]
    if missing_vars:
        print(f"Missing environment variables: {', '.join(missing_vars)}")
        return False
    return True

if check_env_vars():
    credential = DefaultAzureCredential()
    # Test authentication
    try:
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        print("Successfully authenticated using environment credentials!")
    except Exception as e:
        print(f"Authentication failed: {str(e)}")

## 문제 해결 단계

### 1. 역할 할당 확인

```bash
# Check role assignments for your user/service principal
az role assignment list --assignee "your-email@domain.com" --output table
```

### 2. 수집한 토큰 디버깅


In [ ]:
import logging
from azure.identity import DefaultAzureCredential

# Set up detailed logging
logger = logging.getLogger('azure.identity')
logger.setLevel(logging.DEBUG)

# Use a basic StreamHandler instead of LoggingHandler
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

def test_authentication():
    try:
        credential = DefaultAzureCredential(logging_enable=True)
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        print(f"Authentication successful!")
        return True
    except Exception as e:
        print(f"Authentication failed with error: {str(e)}")
        print("\nTroubleshooting steps:")
        print("1. Verify you're logged in: 'az account show'")
        print("2. Check role assignments: 'az role assignment list'")
        print("3. Verify tenant ID: 'az account show --query tenantId'")
        return False

# Run the test
test_authentication()


### 3. Azure AI Developer 역할 확인

In [ ]:
def verify_ai_developer_role(subscription_id, resource_group, resource_name):
    from azure.mgmt.authorization import AuthorizationManagementClient
    
    auth_client = AuthorizationManagementClient(
        credential=DefaultAzureCredential(),
        subscription_id=subscription_id
    )
    
    resource_id = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}"
    
    assignments = auth_client.role_assignments.list_for_scope(resource_id)
    
    ai_developer_role_id = "a97b65f3-24c7-4388-baec-2e87135dc908"  # Azure AI Developer role ID
    
    for assignment in assignments:
        if assignment.role_definition_id.endswith(ai_developer_role_id):
            return True
    
    return False

### 4. Agent Service 권한 추가하기

In [ ]:
# Install required packages
!pip install azure-mgmt-authorization azure-mgmt-resource

In [ ]:
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.resource import ResourceManagementClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
import os, jwt

def assign_roles():
    try:
        # Parse connection string for resource details
        conn_str = os.environ.get("PROJECT_CONNECTION_STRING")
        parts = conn_str.split(';')
        subscription_id = parts[1]
        resource_group = parts[2]
        workspace_name = parts[3]
        
        # Initialize project client
        project_client = AIProjectClient.from_connection_string(
            credential=DefaultAzureCredential(),
            conn_str=conn_str
        )
        print("✅ Created AIProjectClient")
        
        # Initialize credential and get current user's principal ID
        credential = DefaultAzureCredential()
        token = credential.get_token("https://management.azure.com/.default")
        claims = jwt.decode(token.token, options={"verify_signature": False})
        principal_id = claims.get('oid')  # Object ID is the principal ID in Azure AD
        
        # Initialize clients
        auth_client = AuthorizationManagementClient(credential, subscription_id)
        resource_client = ResourceManagementClient(credential, subscription_id)
        
        # Get workspace resource ID
        workspace_scope = (f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/"
                         f"providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}")
        
        # Get search service scope from the connection
        search_conn = project_client.connections.get_default(
            connection_type=ConnectionType.AZURE_AI_SEARCH,
            include_credentials=True
        )
        search_endpoint = search_conn.endpoint_url
        search_name = search_endpoint.split('.')[0].split('//')[1]
        search_scope = (f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/"
                       f"providers/Microsoft.Search/searchServices/{search_name}")

        # Define custom role for agent permissions
        custom_role_name = "AI Foundry Agent Operator"
        custom_role_def = {
            "properties": {  # Added properties wrapper
                "roleName": custom_role_name,
                "description": "Custom role for AI Foundry agent operations",
                "assignableScopes": [workspace_scope],
                "permissions": [{
                    "actions": [
                        "Microsoft.MachineLearning/workspaces/agents/*",  # Updated action format
                        "Microsoft.MachineLearning/workspaces/connections/*"
                    ],
                    "notActions": [],
                    "dataActions": [
                        "Microsoft.MachineLearning/workspaces/agents/*",
                        "Microsoft.MachineLearning/workspaces/connections/*"
                    ],
                    "notDataActions": []
                }],
                "type": "CustomRole"  # Added type
            }
        }

        # Create custom role definition
        try:
            role_definition_id = os.urandom(16).hex()
            custom_role = auth_client.role_definitions.create_or_update(
                scope=workspace_scope,
                role_definition_id=role_definition_id,
                role_definition=custom_role_def
            )
            print(f"✅ Created custom role: {custom_role_name}")
        except Exception as e:
            print(f"⚠️ Custom role creation error: {str(e)}")
            print("Trying alternative role definition format...")
            
            # Try alternative format if first attempt fails
            try:
                custom_role_def_alt = {
                    "properties": {
                        "roleName": custom_role_name,
                        "description": "Custom role for AI Foundry agent operations",
                        "assignableScopes": [workspace_scope],
                        "permissions": [{
                            "actions": [
                                "Microsoft.MachineLearningServices/*"  # Broader permissions as fallback
                            ],
                            "notActions": [],
                            "dataActions": [
                                "Microsoft.MachineLearningServices/*"
                            ],
                            "notDataActions": []
                        }],
                        "type": "CustomRole"
                    }
                }
                
                custom_role = auth_client.role_definitions.create_or_update(
                    scope=workspace_scope,
                    role_definition_id=role_definition_id,
                    role_definition=custom_role_def_alt
                )
                print(f"✅ Created custom role with alternative permissions: {custom_role_name}")
            except Exception as e2:
                print(f"❌ Alternative role creation also failed: {str(e2)}")
        
        # Role definitions to assign
        roles_to_assign = [
            {
                "name": "Azure AI Developer",
                "scope": workspace_scope
            },
            {
                "name": "Storage Blob Data Contributor",
                "scope": workspace_scope
            },
            {
                "name": "Search Service Contributor",
                "scope": search_scope
            },
            {
                "name": custom_role_name,
                "scope": workspace_scope
            }
        ]
        
        print("\n🔑 Starting role assignments...")
        
        for role in roles_to_assign:
            try:
                # Get role definition ID
                role_defs = list(auth_client.role_definitions.list(
                    scope=role["scope"],
                    filter=f"roleName eq '{role['name']}'"
                ))
                if not role_defs:
                    print(f"❌ Could not find role definition for {role['name']}")
                    continue
                    
                role_def_id = role_defs[0].id
                
                # Create role assignment
                assignment = auth_client.role_assignments.create(
                    scope=role["scope"],
                    role_assignment_name=os.urandom(16).hex(),
                    parameters={
                        "properties": {
                            "roleDefinitionId": role_def_id,
                            "principalId": principal_id,
                            "principalType": "User"
                        }
                    }
                )
                print(f"✅ Assigned {role['name']} role")
                
            except Exception as e:
                print(f"❌ Error assigning {role['name']}: {str(e)}")
        
        print("\n⚠️ Note: Role assignments may take a few minutes to propagate")
        
    except Exception as e:
        print(f"❌ Error in role assignment process: {str(e)}")
        import traceback
        print(f"Full traceback:\n{traceback.format_exc()}")

# Execute the role assignments
assign_roles()

## 일반적인 문제 및 해결 방법

1. **토큰을 얻어오지 못했을 때**
   - Azure CLI login 확인: `az login --tenant <tenant-id>`
   - 디폴트 구독 확인: `az account show`
   - 테넌트 확인: `az account set --subscription <subscription-id>`

2. **역할 할당이 안되었는지 확인**
   - Azure AI Developer 역할 추가:
   ```bash
   az role assignment create --assignee "user@domain.com" \
       --role "Azure AI Developer" \
       --scope "/subscriptions/<subscription-id>/resourceGroups/<resource-group>/providers/Microsoft.CognitiveServices/accounts/<resource-name>"
   ```

3. **환경 변수 확인**
   - 환경 변수가 제대로 설정되어 있는지 확인
   - 변수 이름에 오타가 없는지 확인
   - 값에 빈칸이 들어가 있는지 확인

## 모범 사례

1. Service principal 자격 증명에는 항상 환경 변수를 사용
2. 적절한 오류 처리 및 로깅 구현
3. Azure 서비스에 배포할 때 매니지드 아이덴티티 사용
4. Service principal secrets을 정기적으로 교체
5. 역할을 할당할 때 최소 권한 원칙을 따르세요.

## 추가 리소스

- [Azure Identity Documentation](https://docs.microsoft.com/python/api/overview/azure/identity-readme)
- [DefaultAzureCredential Authentication Flow](https://docs.microsoft.com/azure/developer/python/azure-sdk-authenticate)
- [Azure RBAC Documentation](https://docs.microsoft.com/azure/role-based-access-control/overview)